In [ ]:
import json
import re
from pathlib import Path
import traceback




def read_file(filename):
    f = open(filename, 'r')
    return f.read()

def parse_older_format(data_str)->list:    
    line_selector = r"([0-9]{4})-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1]) (2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9]),([0-9]{3}) (.*) b'(.*)'"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        year, month, day, hour, minute, second, ms, importance, data = i
        d = json.loads(data)
        data_list.append(d)
    return data_list

def parse_newer_format(data_str)->list:    
    line_selector = r"\[(Sun|Mon|Tues|Wed|Thu|Fri|Sat), (\d{2}) (Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec) (\d{4}) (\d{2}):(\d{2}):(\d{2})\] INFO b'(.*)'"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        day, date, month, year, hour, minute, second, data = i
        d = json.loads(data)
        data_list.append(d)
    return data_list


def parse_ttn_log_format(data_str)->list:    
    line_selector = r"([0-9]{4})-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1]) (2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9]),([0-9]{3}) INFO  TtnListener:\d+ - Message arrived on topic 'icss_lora_tracker/devices/icspace19/up': ({.*})"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        year, month, day, hour, minute, second, ms, data = i
        d = json.loads(data)
        data_list.append(d)
    return data_list


def special_replace(raw_str,the_string):
    return raw_str.replace("{}:".format(the_string),'"{}":'.format(the_string))

def process_str(string):
    raw_str = string.replace("MSG","").replace("(","{").replace(")","}").replace("=",":").replace("'",'"')
    things_to_replace = ["app_id","dev_id","hardware_serial","counter","payload_raw","payload_fields",
                         "metadata","frequency","modulation","data_rate","airtime","coding_rate",
                        "gtw_id","gtw_trusted","channel","rssi","snr","rf_chain","latitude","longitude",
                         "location_source","analog_in_3","barometric_pressure_0","digital_out_4","gps_2",
                         "temperature_1","altitude","is_retry","antenna"]

    for i in things_to_replace:

        raw_str = raw_str.replace("{}".format(i),'"{}"'.format(i))

    raw_str = raw_str.replace("{time",'{"time"')
    raw_str = raw_str.replace("True",'true')
    raw_str = raw_str.replace("time:",'"time":')
    raw_str = raw_str.replace("fine_timestamp:",'"fine_timestamp":')
    raw_str = raw_str.replace("fine_timestamp_encrypted:",'"fine_timestamp_encrypted":')
    raw_str = raw_str.replace("timestamp:",'"timestamp":')
    raw_str = special_replace(raw_str,"port")
    raw_str = special_replace(raw_str,"gateways")

    return raw_str

def parse_jackson_log_format(data_str)->list:    
    line_selector = r"(MSG\(app_id='icss_lora_tracker', dev_id='icspace.*)"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        data = i
        data = process_str(data)
        if "relative_humidity_2" in data:
            continue
        if "analog_in_1" in data:
            continue

        try:
            d = json.loads(data)
        except Exception:
            print(traceback.format_exc())
            print(data)

        data_list.append(d)
    return data_list





In [ ]:
filename = "mqtt.txt"
str_data= read_file(filename)
data_new = parse_newer_format(str_data)

In [ ]:
icspace24_list = []
for i in data_new:
    dev_id = i.get('dev_id', None)
    if dev_id == "icspace24":
        icspace24_list.append(i)

In [ ]:
import base64
import json
from datetime import datetime
from datetime import timezone


class PacketParser:
    def __init__(self, raw_packet):
        self.raw_packet = raw_packet
        self.current_long = 0
        self.current_lat = 0
        self.current_alt = 0
        self.current_time = 0
        self.num_sats = 0
        self.days_of_playback = 0
        self.noloadVoltage = 0
        self.loadVoltage = 0
        self.pressure = 0
        self.reset_cnt = 0
        self.boardTemp = 0
        self.device_id = ""
        self.past_positions =[]

    def parse_packet(self):
        payload_json = self.raw_packet

        date_time_str = payload_json["metadata"]["time"][:-4]
        self.current_time = datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S.%f')
        self.current_time = self.current_time.replace(tzinfo=timezone.utc)

        self.device_id = payload_json["dev_id"]

        
        try:
            raw_payload = payload_json["payload_raw"]
        except KeyError:
            return

        if raw_payload == None:
            raise ValueError("No payload found")

        try:
            self.parse_raw_payload(raw_payload,self.current_time)
        except Exception:
            print(traceback.format_exc())
            print(data)
        
    def parse_raw_payload(self,raw_payload,current_time):

        hex_payload = base64.b64decode(raw_payload)

        self.pressure = ((hex_payload[2] >> 1) & 0b01111111) * 10
        self.reset_cnt = hex_payload[3] & 0b00000111
        self.boardTemp = int.from_bytes(hex_payload[4:5], byteorder="little", signed=True)
        self.noloadVoltage = ((hex_payload[0] >> 3) & 0b00011111) + 18
        self.loadVoltage = (((hex_payload[0] << 2) & 0b00011100) | ((hex_payload[1] >> 6) & 0b00000011)) + 18
        self.days_of_playback = hex_payload[1] & 0b00111111
        self.num_sats = hex_payload[3] >> 3 & 0b00011111
        self.current_long = int.from_bytes(hex_payload[7:9], byteorder="little", signed=True) * 0xffff / 1e7
        self.current_lat = int.from_bytes(hex_payload[5:7], byteorder="little", signed=True) * 0xffff / 1e7
        self.current_alt = int(int.from_bytes(hex_payload[9:11], byteorder="little", signed=False) * 0xff / 1000)
        
        current_data_size = 11 # Number of bytes to store the current position and temperature/sensor info
        past_data_size = 8    #Number of bytes to store one set of past long,lat,alt, timestamp
        
        self.past_positions = []
        for i in range(13):
            offset = (i+1) * past_data_size
            current_long = int.from_bytes(hex_payload[13+offset:15+offset], byteorder="little", signed=True) * 0xffff / 1e7
            current_lat = int.from_bytes(hex_payload[11+offset:13+offset], byteorder="little", signed=True) * 0xffff / 1e7
            current_alt = int(int.from_bytes(hex_payload[15+offset:17+offset], byteorder="little", signed=False) * 0xff / 1000)
            time = datetime.fromtimestamp(datetime.timestamp(current_time) - int.from_bytes(hex_payload[17+offset:19+offset], byteorder="little", signed=False) * 60)
            
            past_pos = {
                "altitude":current_alt,
                "latitude":current_lat,
                "longitude":current_long,
                "time_stamp":time

            }
            
            self.past_positions.append(past_pos)


In [ ]:
for tx in icspace24_list:
    parsed_packet = PacketParser(tx)
    try:
        parsed_packet.parse_packet()
        
        payload_fields = {"barometric_pressure_0":parsed_packet.pressure,
                          "digital_out_4":parsed_packet.num_sats,
                          "gps_2":{"altitude":parsed_packet.current_alt,"latitude":parsed_packet.current_lat,"longitude":parsed_packet.current_long},
                          "temperature_1":parsed_packet.boardTemp
                          }
        tx["payload_fields"] = payload_fields
        for past_pos in parsed_packet.past_positions:
            #print(past_pos)
            new_pos = {
                "app_id":"icss_lora_tracker",
                "dev_id": "icspace24",
                "payload_fields":{"gps_2":{
                    "altitude":past_pos["altitude"],
                    "latitude":past_pos["latitude"],
                    "longitude":past_pos["longitude"]
                }},
                "metadata":{"time":past_pos["time_stamp"].isoformat() + 'Z'}
            }
            icspace24_list.append(new_pos)
    except Exception:
        print(traceback.format_exc())
    

In [ ]:
icspace24_list

In [ ]:
# {'app_id': 'icss_lora_tracker',
#   'dev_id': 'icspace23',
#   'hardware_serial': '0093BECA9134091B',
#   'port': 99,
#   'counter': 324,
#   'payload_raw': 'AHMmzgFnAQMCiAAAAAAAAAAAAAMCICEEAQA=',
#   'payload_fields': {'barometric_pressure_0': 190,
#    'digital_out_4': 25,
#    'gps_2': {'altitude': 34, 'latitude': 2.3527065, 'longitude': 3.3750525},
#    'temperature_1': 1},
#   'metadata': {'time': '2020-10-21T15:39:20.984695118Z',
#    'frequency': 868.1,
#    'modulation': 'LORA',
#    'data_rate': 'SF8BW125',
#    'airtime': 154112000,
#    'coding_rate': '4/5',
#    'gateways': [{'gtw_id': 'eui-58a0cbfffe800f4d',
#      'timestamp': 3259604836,
#      'time': '2020-10-21T15:39:20.967922925Z',
#      'channel': 0,
#      'rssi': -91,
#      'snr': 12.25,
#      'rf_chain': 0}]}},

import pandas as pd
import datetime

positions = []
for i in icspace24_list:
    try:
        timepos = i["payload_fields"]["gps_2"]
        timepos["timestamp"] = pd.to_datetime(i["metadata"]["time"].replace("Z", "+00:00"))
        timepos["temperature"] = i["payload_fields"]["temperature_1"]
        positions.append(timepos)
    except Exception:
        print(traceback.format_exc())    
    
def limit_date_range(df):
    return df[(df['timestamp']> "2021-1-22") & (df['timestamp']< "2021-1-23")]

positions_df = pd.DataFrame.from_dict(positions)
positions_df = positions_df[positions_df["altitude"]>60]
positions_df = limit_date_range(positions_df)
positions_df = positions_df.sort_values(by=['timestamp'])
positions_df.to_csv("icspace24.csv")
positions_df

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=positions_df["timestamp"], y=positions_df["altitude"],
                    mode='lines+markers',
                    name='Flight altitude',
                    ))

# Edit the layout
fig.update_layout(title='Altitude[m] vs Time',
                   xaxis_title='Time',
                   yaxis_title='Altitude[m]')


fig.show()
positions_df.iloc[[-1]]

In [ ]:
fig = px.scatter_mapbox(positions_df, lat="latitude", lon="longitude", hover_data=["timestamp", "altitude"])
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
from pymongo import MongoClient
import urllib.parse

# init mongo connection
username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)

flight_data_collection = client["flight_data"]["all_flights"]

In [ ]:
flight_data_collection.insert_many(icspace24_list)

In [ ]:
#client["flight_data"]["raw_logs"].insert_many(jackson_processed)